In [36]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
import matplotlib.pyplot as plt
import time

In [37]:
digits = datasets.load_digits()
features = digits['data']
target = digits['target'] 

In [38]:
train_x, test_x, train_y, test_y = train_test_split(features, target, test_size=0.33)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(1203, 64)
(1203,)
(594, 64)
(594,)


# Pytorch

In [39]:
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import DataLoader, TensorDataset

In [40]:
# 格式转化
pytorch_train_x = torch.tensor(train_x, dtype=torch.float32)
pytorch_train_y = torch.tensor(train_y, dtype=torch.long)

pytorch_test_x = torch.tensor(test_x, dtype=torch.float32)
pytorch_test_y = torch.tensor(test_y, dtype=torch.long)

# 装进数据容器分批
data = TensorDataset(pytorch_train_x, pytorch_train_y)
dataset = DataLoader(data, 64)

In [41]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 128)
        self.fc6 = nn.Linear(128, 64)
        self.fc7 = nn.Linear(64, 10)
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.relu(self.fc5(x))
        x = self.relu(self.fc6(x))
        result = self.fc7(x)
        return result
    
    def predict(self, x):
        result = self.softmax(self(x)).sort()[1][:,-1]
        return result

### CPU Version

In [42]:
model = Model()
optimizer = optim.Adam(model.parameters())
loss_fun = nn.CrossEntropyLoss()

In [43]:
%%time
# 训练
for i in range(100):
    loss_record = 0
    start_time = time.time()
    
    for batch, (x, y) in enumerate(dataset):
        optimizer.zero_grad()
        y_pre = model(x)
        
        loss = loss_fun(y_pre, y)
        loss.backward()
        optimizer.step()
        
        loss_record += loss.item()
    
    duration = time.time() - start_time
    
    predict = model.predict(pytorch_test_x).numpy()
    acc_score = accuracy_score(predict, pytorch_test_y.numpy())
    
    print('Epoch: {}, time: {:.3f}s, Loss: {:.3f}, Acc: {:.2f}%'.format(i+1, duration, loss_record/batch, acc_score*100))

Epoch: 1, time: 0.282s, Loss: 1.881, Acc: 69.70%
Epoch: 2, time: 0.132s, Loss: 0.646, Acc: 89.90%
Epoch: 3, time: 0.137s, Loss: 0.283, Acc: 93.60%
Epoch: 4, time: 0.152s, Loss: 0.150, Acc: 93.27%
Epoch: 5, time: 0.126s, Loss: 0.145, Acc: 92.59%
Epoch: 6, time: 0.132s, Loss: 0.110, Acc: 95.12%
Epoch: 7, time: 0.139s, Loss: 0.054, Acc: 95.45%
Epoch: 8, time: 0.134s, Loss: 0.034, Acc: 94.61%
Epoch: 9, time: 0.122s, Loss: 0.033, Acc: 96.80%
Epoch: 10, time: 0.124s, Loss: 0.025, Acc: 96.13%
Epoch: 11, time: 0.133s, Loss: 0.018, Acc: 96.13%
Epoch: 12, time: 0.120s, Loss: 0.034, Acc: 94.11%
Epoch: 13, time: 0.120s, Loss: 0.032, Acc: 95.79%
Epoch: 14, time: 0.116s, Loss: 0.015, Acc: 96.30%
Epoch: 15, time: 0.117s, Loss: 0.006, Acc: 97.31%
Epoch: 16, time: 0.118s, Loss: 0.004, Acc: 96.13%
Epoch: 17, time: 0.121s, Loss: 0.008, Acc: 97.31%
Epoch: 18, time: 0.117s, Loss: 0.008, Acc: 96.30%
Epoch: 19, time: 0.126s, Loss: 0.018, Acc: 95.45%
Epoch: 20, time: 0.124s, Loss: 0.031, Acc: 94.44%
Epoch: 21

### GPU Version

In [44]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pytorch_test_x, pytorch_test_y = pytorch_test_x.to(device), pytorch_test_y.to(device)
model = Model().to(device)
optimizer = optim.Adam(model.parameters())
loss_fun = nn.CrossEntropyLoss()

In [45]:
%%time
# 训练
for i in range(100):
    loss_record = 0
    start_time = time.time()
    
    for batch, (x, y) in enumerate(dataset):
        optimizer.zero_grad()

        x, y = x.to(device), y.to(device)
        y_pre = model(x)
        
        loss = loss_fun(y_pre, y)
        loss.backward()
        optimizer.step()
        
        loss_record += loss.cpu().item()
    
    duration = time.time() - start_time
    
    predict = model.predict(pytorch_test_x).cpu().numpy()
    acc_score = accuracy_score(predict, pytorch_test_y.cpu().numpy())
    
    print('Epoch: {}, time: {:.3f}s, Loss: {:.3f}, Acc: {:.2f}%'.format(i+1, duration, loss_record/batch, acc_score*100))

Epoch: 1, time: 0.073s, Loss: 1.852, Acc: 77.44%
Epoch: 2, time: 0.069s, Loss: 0.503, Acc: 85.86%
Epoch: 3, time: 0.070s, Loss: 0.354, Acc: 92.76%
Epoch: 4, time: 0.071s, Loss: 0.178, Acc: 94.95%
Epoch: 5, time: 0.070s, Loss: 0.102, Acc: 94.95%
Epoch: 6, time: 0.068s, Loss: 0.071, Acc: 94.44%
Epoch: 7, time: 0.071s, Loss: 0.053, Acc: 95.96%
Epoch: 8, time: 0.074s, Loss: 0.041, Acc: 95.12%
Epoch: 9, time: 0.076s, Loss: 0.047, Acc: 94.95%
Epoch: 10, time: 0.073s, Loss: 0.027, Acc: 96.30%
Epoch: 11, time: 0.073s, Loss: 0.019, Acc: 96.63%
Epoch: 12, time: 0.070s, Loss: 0.022, Acc: 95.29%
Epoch: 13, time: 0.075s, Loss: 0.011, Acc: 95.45%
Epoch: 14, time: 0.071s, Loss: 0.009, Acc: 95.45%
Epoch: 15, time: 0.069s, Loss: 0.005, Acc: 95.79%
Epoch: 16, time: 0.076s, Loss: 0.011, Acc: 96.30%
Epoch: 17, time: 0.069s, Loss: 0.004, Acc: 96.46%
Epoch: 18, time: 0.070s, Loss: 0.012, Acc: 96.13%
Epoch: 19, time: 0.078s, Loss: 0.007, Acc: 96.63%
Epoch: 20, time: 0.071s, Loss: 0.006, Acc: 97.14%
Epoch: 21